In [1]:
import pandas as pd
import numpy as np

# Carichiamo il file Excel
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Data')

# Vediamo cosa c'è dentro
print(f"Numero totale di pazienti: {len(df)}")
print(f"\nNumero di colonne: {len(df.columns)}")

Numero totale di pazienti: 96

Numero di colonne: 29


In [2]:
# Colonne necessarie per la formula SRK/T
colonne_necessarie = ['Bio-AL', 'Bio-Ks', 'Bio-Kf', 'IOL Power', 'A-Constant', 'PostOP Spherical Equivalent']

# Verifichiamo che ci siano tutte
for col in colonne_necessarie:
    if col in df.columns:
        print(f"✓ {col} trovata")
    else:
        print(f"✗ {col} NON TROVATA!")

# Vediamo un esempio di dati
print("\nEsempio dei primi 3 pazienti:")
print(df[colonne_necessarie].head(3))

✓ Bio-AL trovata
✓ Bio-Ks trovata
✓ Bio-Kf trovata
✓ IOL Power trovata
✓ A-Constant trovata
✓ PostOP Spherical Equivalent trovata

Esempio dei primi 3 pazienti:
   Bio-AL  Bio-Ks  Bio-Kf  IOL Power  A-Constant  PostOP Spherical Equivalent
0   26.40   44.00   43.50       15.5       119.1                       -3.875
1   23.59   49.41   46.68       15.0       118.4                       -3.125
2   23.25   45.67   45.30       22.5       120.2                       -3.000


In [3]:
# Calcolo del K medio
df['K_avg'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2

# Vediamo cosa abbiamo calcolato
print("K medio per i primi 5 pazienti:")
print(df[['Bio-Ks', 'Bio-Kf', 'K_avg']].head())

# Statistiche del K medio
print(f"\nStatistiche K medio:")
print(f"Minimo: {df['K_avg'].min():.2f} D")
print(f"Massimo: {df['K_avg'].max():.2f} D") 
print(f"Media: {df['K_avg'].mean():.2f} D")

K medio per i primi 5 pazienti:
   Bio-Ks  Bio-Kf   K_avg
0   44.00   43.50  43.750
1   49.41   46.68  48.045
2   45.67   45.30  45.485
3   44.23   43.38  43.805
4   47.70   44.60  46.150

Statistiche K medio:
Minimo: 39.17 D
Massimo: 48.05 D
Media: 43.44 D


In [4]:
def calcola_refrazione_srkt(AL, K_avg, IOL_power, A_constant):
    """
    Formula SRK/T ORIGINALE - senza correzione per imaginary ACD
    Restituirà NaN per alcuni pazienti
    """
    
    # COSTANTI
    na = 1.336   
    nc = 1.333   
    ncm1 = 0.333 
    V = 12       
    
    # STEP 1: Raggio corneale
    r = 337.5 / K_avg
    
    # STEP 2: Lunghezza assiale corretta
    if AL <= 24.2:
        LCOR = AL  
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL  
    
    # STEP 3: Larghezza corneale
    Cw = -5.41 + 0.58412 * LCOR + 0.098 * K_avg
    
    # STEP 4: Altezza corneale (SENZA correzione)
    H = r - np.sqrt(r * r - (Cw * Cw / 4))  # Darà NaN se negativo
    
    # STEP 5-8: Resto del calcolo
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336
    ACD_est = H + offset
    
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK
    
    numeratore = (1000 * na * (na * r - ncm1 * LOPT) - 
                  IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominatore = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                    0.001 * IOL_power * (LOPT - ACD_est) * 
                    (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numeratore / denominatore

# Applica la formula originale
df['Refrazione_Prevista_SRKT'] = df.apply(
    lambda row: calcola_refrazione_srkt(
        AL=row['Bio-AL'], 
        K_avg=row['K_avg'], 
        IOL_power=row['IOL Power'], 
        A_constant=row['A-Constant']
    ), 
    axis=1
)

# Calcola errori SOLO per i pazienti validi (no NaN)
validi = df['Refrazione_Prevista_SRKT'].notna()
df_validi = df[validi].copy()

df_validi['Errore_Predizione'] = df_validi['PostOP Spherical Equivalent'] - df_validi['Refrazione_Prevista_SRKT']
df_validi['Errore_Assoluto'] = abs(df_validi['Errore_Predizione'])

# Risultati
print(f"RISULTATI SRK/T (solo pazienti validi):")
print("="*50)
print(f"Pazienti validi: {len(df_validi)} su {len(df)} totali")
print(f"Pazienti esclusi (NaN): {len(df) - len(df_validi)}")
print(f"\nMAE (Mean Absolute Error): {df_validi['Errore_Assoluto'].mean():.3f} D")
print(f"Errore medio: {df_validi['Errore_Predizione'].mean():.3f} D")
print(f"Deviazione standard: {df_validi['Errore_Predizione'].std():.3f} D")

RISULTATI SRK/T (solo pazienti validi):
Pazienti validi: 67 su 96 totali
Pazienti esclusi (NaN): 29

MAE (Mean Absolute Error): 1.274 D
Errore medio: 0.165 D
Deviazione standard: 1.762 D


C:\Users\offic\AppData\Local\Temp\ipykernel_24872\879622920.py:26: RuntimeWarning: invalid value encountered in sqrt
  H = r - np.sqrt(r * r - (Cw * Cw / 4))  # Darà NaN se negativo


Ora ho bisogno di trovare indici refrattivi e cheratometrici ottimizzati e poi di usare machine learning per altre variabili. poi va provato con diversi semi